In [1]:
import ipywidgets
import traitlets
import yt

/home/mturk/conda-py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")

In [3]:
def display_arr(arr):
    unit_registry = arr.units.registry
    dimension = arr.units.dimensions
    equiv = [k for k, v in unit_registry.lut.items() if v[1] is dimension]
    dropdown = ipywidgets.Dropdown(options = sorted(equiv), value = str(arr.units))
    def arr_updater(arr, texts):
        def _value_updater(change):
            arr2 = arr.in_units(change['new'])
            if arr2.shape == ():
                arr2 = [arr2]
            for v, t in zip(arr2, texts):
                t.value = str(v.value)
        return _value_updater
    if arr.shape == ():
        arr_iter = [arr]
    else:
        arr_iter = arr
    texts = [ipywidgets.Text(value = str(_.value), disabled = True) for _ in arr_iter]
    dropdown.observe(arr_updater(arr, texts), names="value")
    return ipywidgets.HBox(texts + [dropdown])

In [4]:
display_arr(ds.domain_width)

HBox(children=(Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length')))

In [5]:
def display_ds(ds):
    attrs = ["domain_width", "domain_left_edge", "domain_right_edge", "current_time"]
    lb = ipywidgets.VBox([ ipywidgets.Label(value = _) for _ in attrs])
    vb = ipywidgets.VBox([ display_arr(getattr(ds, _)) for _ in attrs])
    hb = ipywidgets.HBox([lb, vb])
    return hb

In [6]:
display_ds(ds)

HBox(children=(VBox(children=(Label(value='domain_width'), Label(value='domain_left_edge'), Label(value='domain_right_edge'), Label(value='current_time'))), VBox(children=(HBox(children=(Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length'))), HBox(children=(Text(value='0.0', disabled=True), Text(value='0.0', disabled=True), Text(value='0.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length'))), HBox(children=(Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length'))), HBox(children=(Text(value='0.0060000200028298', disabled=True), Dropdown(options=('code_time', 'd', 'day', 'hr', 'min', 's', 't_geom', 't_pl', 'yr'), value='code_time')))))))

In [7]:
import ipywidgets as widgets # Loads the Widget framework.
from IPython.core.magics.namespace import NamespaceMagics # Used to query namespace.

# For this example, hide these names, just to avoid polluting the namespace further
get_ipython().user_ns_hidden['widgets'] = widgets
get_ipython().user_ns_hidden['NamespaceMagics'] = NamespaceMagics

In [8]:
class VariableInspectorWindow(object):
    instance = None

    def __init__(self, ipython):
        """Public constructor."""
        if VariableInspectorWindow.instance is not None:
            raise Exception("""Only one instance of the Variable Inspector can exist at a
                time.  Call close() on the active instance before creating a new instance.
                If you have lost the handle to the active instance, you can re-obtain it
                via `VariableInspectorWindow.instance`.""")

        VariableInspectorWindow.instance = self
        self.closed = False
        self.namespace = NamespaceMagics()
        self.namespace.shell = ipython.kernel.shell

        self._box = widgets.Box()
        self._box.layout.overflow_y = 'scroll'
        self._table = widgets.HTML(value = 'Not hooked')
        self._box.children = [self._table]

        self._ipython = ipython
        self._ipython.events.register('post_run_cell', self._fill)

    def close(self):
        """Close and remove hooks."""
        if not self.closed:
            self._ipython.events.unregister('post_run_cell', self._fill)
            self._box.close()
            self.closed = True
            VariableInspectorWindow.instance = None

    def _fill(self):
        """Fill self with variable information."""
        values = self.namespace.who_ls()
        self._table.value = '<div class="rendered_html jp-RenderedHTMLCommon"><table><thead><tr><th>Name</th><th>Type</th><th>Value</th></tr></thead><tr><td>' + \
            '</td></tr><tr><td>'.join(['{0}</td><td>{1}</td><td>{2}'.format(v, type(eval(v)).__name__, str(eval(v))) for v in values]) + \
            '</td></tr></table></div>'

    def _ipython_display_(self):
        """Called when display() or pyout is used to display the Variable
        Inspector."""
        self._box._ipython_display_()


In [ ]:
inspector = VariableInspectorWindow(get_ipython())
inspector

In [10]:
v = 'this'

In [11]:
ds.fields

Parsing Hierarchy : 100%|██████████| 173/173 [00:00<00:00, 3389.44it/s]


In [12]:
ds.fields.gas.density.sampling_type

'cell'

In [13]:
dir(ds.fields.gas.density)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_copy_def',
 '_function',
 '_ion_to_label',
 '_is_ion',
 '_unit_registry',
 'check_available',
 'dimensions',
 'display_field',
 'display_name',
 'ds',
 'get_dependencies',
 'get_label',
 'get_latex_display_name',
 'get_projected_units',
 'get_source',
 'get_units',
 'name',
 'nodal_flag',
 'not_in_all',
 'output_units',
 'particle_type',
 'sampling_type',
 'take_log',
 'unit_registry',
 'units',
 'validators',
 'vector_field']

In [14]:
density = ds.fields.gas.density

In [15]:
import inspect

In [16]:
inspect.getmembers(density._function)

[('__annotations__', {}),
 ('__call__',
  <method-wrapper '__call__' of function object at 0x7f4bc02e1488>),
 ('__class__', function),
 ('__closure__', (<cell at 0x7f4bc02cc3d8: tuple object at 0x7f4bc0315f88>,)),
 ('__code__',
  <code object _TranslationFunc at 0x7f4bd00eaed0, file "/home/mturk/yt/yt/yt/fields/derived_field.py", line 40>),
 ('__defaults__', None),
 ('__delattr__',
  <method-wrapper '__delattr__' of function object at 0x7f4bc02e1488>),
 ('__dict__', {'alias_name': ('enzo', 'Density')}),
 ('__dir__', <function function.__dir__>),
 ('__doc__', None),
 ('__eq__', <method-wrapper '__eq__' of function object at 0x7f4bc02e1488>),
 ('__format__', <function function.__format__>),
 ('__ge__', <method-wrapper '__ge__' of function object at 0x7f4bc02e1488>),
 ('__get__', <method-wrapper '__get__' of function object at 0x7f4bc02e1488>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of function object at 0x7f4bc02e1488>),
 ('__globals__',
  {'DerivedField': yt.fields.d

In [18]:
inspect.getclosurevars(density._function)

ClosureVars(nonlocals={'field_name': ('enzo', 'Density')}, globals={}, builtins={}, unbound={'copy'})

In [19]:
def display_field_source(fobj):
    source = ipywidgets.Textarea(value = fobj.get_source())
    ac = ipywidgets.Accordion([source])
    ac.set_title(0, "Source")
    return ac

In [20]:
ac = display_field_source(density)

In [22]:
ac.children

(Textarea(value='    def _TranslationFunc(field, data):\n        # We do a bunch of in-place modifications, so we will copy this.\n        return data[field_name].copy()\n'),)

In [26]:
ac.children[0].style.width = '100%'

In [27]:
ac

Accordion(children=(Textarea(value='    def _TranslationFunc(field, data):\n        # We do a bunch of in-place modifications, so we will copy this.\n        return data[field_name].copy()\n'),), _titles={'0': 'Source'})

In [28]:
import pygments

In [30]:
ds.fields.field_types

{'all', 'deposit', 'enzo', 'gas', 'index', 'io'}

In [124]:
def display_all_fields(ds):
    fnames = []
    children = []
    for ftype in sorted(ds.fields.field_types):
        fnc = getattr(ds.fields, ftype)
        names = dir(fnc)
        names.sort()
        def change_field(_ftype, _box, _var_window):
            def _change_field(event):
                fobj = getattr(_ftype, event['new'])
                _box.value = fobj.get_source()
                values = inspect.getclosurevars(fobj._function).nonlocals
                _var_window.value = '<div class="rendered_html jp-RenderedHTMLCommon"><table><thead><tr><th>Name</th><th>Type</th><th>Value</th></tr></thead><tr><td>' + \
            '</td></tr><tr><td>'.join(['{0}</td><td>{1}</td><td>{2}'.format(v, type(values[v]).__name__, str(values[v])) for v in sorted(values)]) + \
            '</td></tr></table></div>'
            return _change_field
        flist = ipywidgets.Select(options = names, layout = ipywidgets.Layout(height = '95%'))
        source = ipywidgets.Textarea(layout = ipywidgets.Layout(width = '100%', height = '8em'))
        var_window = ipywidgets.HTML(value = 'Empty')
        var_box = ipywidgets.Box(layout = ipywidgets.Layout(width = '100%', height = '100%', overflow_y = 'scroll'))
        var_box.children = [var_window]
        ftype_tabs = ipywidgets.Tab(children = [source, var_box], layout = ipywidgets.Layout(flex = '2 1 auto', width = 'auto', height = '95%'))
        ftype_tabs.set_title(0, "Source")
        ftype_tabs.set_title(1, "Variables")
        flist.observe(change_field(fnc, source, var_window), "value")
        children.append(ipywidgets.HBox([flist, ftype_tabs], layout = ipywidgets.Layout(height = '14em')))
        fnames.append(ftype)
    tabs = ipywidgets.Tab(children = children)
    for i, n in enumerate(fnames):
        tabs.set_title(i, n)
    return tabs

In [125]:
display_all_fields(ds)

Tab(children=(HBox(children=(Select(layout=Layout(height='95%'), options=('age', 'creation_time', 'dynamical_time', 'mesh_id', 'metallicity_fraction', 'particle_angular_momentum', 'particle_angular_momentum_magnitude', 'particle_angular_momentum_x', 'particle_angular_momentum_y', 'particle_angular_momentum_z', 'particle_cylindrical_velocity_theta', 'particle_cylindrical_velocity_z', 'particle_index', 'particle_mass', 'particle_ones', 'particle_position', 'particle_position_cylindrical_radius', 'particle_position_cylindrical_theta', 'particle_position_cylindrical_z', 'particle_position_relative', 'particle_position_relative_x', 'particle_position_relative_y', 'particle_position_relative_z', 'particle_position_spherical_phi', 'particle_position_spherical_radius', 'particle_position_spherical_theta', 'particle_position_x', 'particle_position_y', 'particle_position_z', 'particle_radial_velocity', 'particle_radius', 'particle_specific_angular_momentum', 'particle_specific_angular_momentum_x', 'particle_specific_angular_momentum_y', 'particle_specific_angular_momentum_z', 'particle_spherical_position_phi', 'particle_spherical_position_radius', 'particle_spherical_position_theta', 'particle_spherical_velocity_phi', 'particle_spherical_velocity_radius', 'particle_spherical_velocity_theta', 'particle_type', 'particle_velocity', 'particle_velocity_cylindrical_radius', 'particle_velocity_cylindrical_theta', 'particle_velocity_cylindrical_z', 'particle_velocity_magnitude', 'particle_velocity_relative', 'particle_velocity_relative_x', 'particle_velocity_relative_y', 'particle_velocity_relative_z', 'particle_velocity_spherical_phi', 'particle_velocity_spherical_radius', 'particle_velocity_spherical_theta', 'particle_velocity_x', 'particle_velocity_y', 'particle_velocity_z'), value='age'), Tab(children=(Textarea(value='', layout=Layout(height='8em', width='100%')), Box(children=(HTML(value='Empty'),), layout=Layout(height='100%', overflow_y='scroll', width='100%'))), layout=Layout(flex='2 1 auto', height='95%', width='auto'), _titles={'0': 'Source', '1': 'Variables'})), layout=Layout(height='14em')), HBox(children=(Select(layout=Layout(height='95%'), options=('all_cic', 'all_cic_age', 'all_cic_velocity_x', 'all_cic_velocity_y', 'all_cic_velocity_z', 'all_count', 'all_density', 'all_mass', 'all_nn_age', 'all_nn_velocity_x', 'all_nn_velocity_y', 'all_nn_velocity_z', 'io_cic', 'io_cic_age', 'io_cic_velocity_x', 'io_cic_velocity_y', 'io_cic_velocity_z', 'io_count', 'io_density', 'io_mass', 'io_nn_age', 'io_nn_velocity_x', 'io_nn_velocity_y', 'io_nn_velocity_z'), value='all_cic'), Tab(children=(Textarea(value='', layout=Layout(height='8em', width='100%')), Box(children=(HTML(value='Empty'),), layout=Layout(height='100%', overflow_y='scroll', width='100%'))), layout=Layout(flex='2 1 auto', height='95%', width='auto'), _titles={'0': 'Source', '1': 'Variables'})), layout=Layout(height='14em')), HBox(children=(Select(layout=Layout(height='95%'), options=('Average_creation_time', 'Bx', 'By', 'Bz', 'Cooling_Time', 'Dark_Matter_Density', 'Density', 'Electron_Density', 'Forming_Stellar_Mass_Density', 'Galaxy1Colour', 'Galaxy2Colour', 'HII_Density', 'HI_Density', 'HeIII_Density', 'HeII_Density', 'HeI_Density', 'MBHColour', 'Metal_Density', 'PhiField', 'Phi_pField', 'SFR_Density', 'Star_Particle_Density', 'Temperature', 'TotalEnergy', 'cell_volume', 'dx', 'dy', 'dz', 'gammaHI', 'kphHI', 'kphHeI', 'kphHeII', 'path_element_x', 'path_element_y', 'path_element_z', 'vertex_x', 'vertex_y', 'vertex_z', 'x', 'x-velocity', 'y', 'y-velocity', 'z', 'z-velocity'), value='Average_creation_time'), Tab(children=(Textarea(value='', layout=Layout(height='8em', width='100%')), Box(children=(HTML(value='Empty'),), layout=Layout(height='100%', overflow_y='scroll', width='100%'))), layout=Layout(flex='2 1 auto', height='95%', width='auto'), _titles={'0': 'Source', '1': 'Variables'})), layout=Layout(height='14em')), HBox(children=(Select(layout=Layout(height='95%'), options=('E

In [48]:
ta = ipywidgets.Textarea()

In [90]:
var_window = ipywidgets.HTML()
fobj = ds.fields.gas.density
values = inspect.getclosurevars(fobj._function).nonlocals
var_window.value = '<div class="rendered_html jp-RenderedHTMLCommon"><table><thead><tr><th>Name</th><th>Type</th><th>Value</th></tr></thead><tr><td>' + \
            '</td></tr><tr><td>'.join(['{0}</td><td>{1}</td><td>{2}'.format(v, type(values[v]).__name__, str(values[v])) for v in sorted(values)]) + \
            '</td></tr></table></div>'
var_window

HTML(value='<div class="rendered_html jp-RenderedHTMLCommon"><table><thead><tr><th>Name</th><th>Type</th><th>Value</th></tr></thead><tr><td>field_name</td><td>tuple</td><td>(\'enzo\', \'Density\')</td></tr></table></div>')

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_fields',
 '_make',
 '_replace',
 '_source',
 'builtins',
 'count',
 'globals',
 'index',
 'nonlocals',
 'unbound']